In [1]:
import os
import pandas as pd
from arosics import COREG, COREG_LOCAL
from zipfile import ZipFile
from IPython.display import Image
from rasterio.features import bounds, dataset_features
import sys
sys.path.insert(1, '../')
from pyproj import Proj
from utils import *

%matplotlib inline

* Time series handling is limited. mainly pairwise. Need coding to loop trhough series.
* Takes 30 secs to match two full res S2 scenes in global mode
* Takse about 3 mins to match two full res S2 scenes with 120 grid points and running on 96 cores! (A bit concerning)
* Also running in full res for local matching is very resource intensive!
* Documentation not very straightforward!
* Seems to be partially handaling the cloud cover specially for smaller images which seems to be doing good.
* AU scenes are already co-registered mostly.
* Local matching will fail for small overlaps.
* window size of (500, 500) recommended. 
* overall both S1 and S2 register good over AU
* local matching is recommended
* seems to work better on band 2 (G) for the Antarctica
* tricky to use on large images. works well on downsampled ones.
* Phase correlation does not work when rotation required. It only works for translation shifts. 

In [2]:
DATA_DIR = "../data/files_list"

S1_PRODUCTS = ["SLC", "GRD"]
S2_PRODUCTS = ["L1C", "L2A"]

s1_au_df = pd.read_csv(os.path.join(DATA_DIR, "s1_au.csv"), names=["ID","Path"])
s1_an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID","Path"])

s2_au_df = pd.read_csv(os.path.join(DATA_DIR, "s2_au.csv"), names=["ID","Path"])
s2_an_df = pd.read_csv(os.path.join(DATA_DIR, "s2_an.csv"), names=["ID","Path"])

s1_au_slc_dict = get_scenes_dict(s1_au_df, ["SLC"])
s1_au_grd_dict = get_scenes_dict(s1_au_df, ["GRD"])

s1_an_slc_dict = get_scenes_dict(s1_an_df, ["SLC"])
s1_an_grd_dict = get_scenes_dict(s1_an_df, ["GRD"])

s2_au_l1c_dict = get_scenes_dict(s2_au_df, ["L1C"], False)
s2_au_l2a_dict = get_scenes_dict(s2_au_df, ["L2A"], False)

s2_an_l1c_dict = get_scenes_dict(s2_an_df, ["L1C"], False)
s2_an_l2a_dict = get_scenes_dict(s2_an_df, ["L2A"], False)


In [3]:
shutil.rmtree("../data/inputs/", ignore_errors=True)
# shutil.rmtree("../data/outputs/", ignore_errors=True)
os.makedirs("../data/inputs/")
os.makedirs("../data/outputs/", exist_ok = True)

##### Test two S2 L2A images for AN

In [106]:

id = list(s2_an_l2a_dict.keys())[1]
s2_an_l2a_secne_files = s2_an_l2a_dict[id]

# get the Jan 2023 and Dec 2024 files
s2_an_l2a_secne_pair = [s2_an_l2a_secne_files[10], s2_an_l2a_secne_files[-1]]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
for i, zip_file_path in enumerate(s2_an_l2a_secne_pair):
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/sub{i}")
ref_image_dir = os.path.join("../data/inputs/", id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L2A" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

tgt_image_dir = os.path.join("../data/inputs/", id, f"sub{1}")
tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
tgt_tci_files = list(filter(lambda f: "L2A" in f, tgt_tci_files))
if len(tgt_tci_files) > 1:
    tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
else:
    tgt_image = tgt_tci_files[0]

global_output = os.path.join("../data/outputs/", "L2A_pair", f"{id}_global.tiff")
local_output = os.path.join("../data/outputs/", "L2A_pair", f"{id}_local.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=tgt_image,
    im_tgt=ref_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=tgt_image,
    im_tgt=ref_image,
    grid_res=1000,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    # r_b4match=2,
    # s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()

In [ ]:
datasets_paths = [tgt_image, ref_image,  local_output]
dataset_titles = ["Reference", "Target", "Local matching"]
make_difference_gif(datasets_paths, "s2_l2a_pair.gif", dataset_titles, 0.1)
Image(url='s2_l2a_pair.gif', width = 400, height=400) 

##### Test two AN S2 L1C secenes

In [169]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]
s2_an_l1c_secne_files = s2_an_l1c_r031_dict[id]

# get the Jan 2023 and Dec 2024 files
s2_an_l1c_secne_pair = [s2_an_l1c_secne_files[15], s2_an_l1c_secne_files[5]]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
for i, zip_file_path in enumerate(s2_an_l1c_secne_pair):
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/sub{i}")
ref_image_dir = os.path.join("../data/inputs/", id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

tgt_image_dir = os.path.join("../data/inputs/", id, f"sub{1}")
tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
tgt_tci_files = list(filter(lambda f: "L1C" in f, tgt_tci_files))
if len(tgt_tci_files) > 1:
    tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
else:
    tgt_image = tgt_tci_files[0]

local_output = os.path.join("../data/outputs/", "L1C_AN_pair", f"{id}_local.tiff")
global_output = os.path.join("../data/outputs/", "L1C_AN_pair", f"{id}_global.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=ref_image,
    im_tgt=tgt_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    ws = (1500, 1500),
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=ref_image,
    im_tgt=tgt_image,
    grid_res=1000,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()

In [ ]:
datasets_paths = [ref_image, tgt_image, local_output]
datasets_titles = ["Reference", "Target", "Local matching"]
make_difference_gif(datasets_paths, "s2_l1c_an_pair.gif", datasets_titles, 0.1)
Image(url='s2_l1c_an_pair.gif', width = 400, height=400) 

##### S2 L1C and S1 RTC AN

In [3]:
# find scenes
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
s2_id = list(s2_an_l1c_r031_dict.keys())[0]
s2_an_l1c_secne_files = s2_an_l1c_r031_dict[s2_id]

# shutil.rmtree(f"../data/inputs/{s2_id}/", ignore_errors=True)

# get the Jan 2023 for S2 L1C
s2_an_l1c_scene = s2_an_l1c_secne_files[15]
with ZipFile(s2_an_l1c_scene) as f:
    f.extractall(f"../data/inputs/{s2_id}/sub0")

ref_image_dir = os.path.join("../data/inputs/", s2_id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

ref = rasterio.open(ref_image)
ref_bounds = ref.bounds
ref_crs = ref.crs

ref_proj = Proj(**ref_crs.data)

west, south = ref_proj(ref_bounds.left, ref_bounds.bottom, inverse = True)
east, north = ref_proj(ref_bounds.right, ref_bounds.top, inverse = True)

bbox = f"{west},{south},{east},{north}"

os.makedirs("../data/asf/sub1", exist_ok=True)
with open(f"../data/asf/sub1/bbox.txt", "w") as f:
    f.write(f"{bbox}\n")
    f.write(f"{s2_id}\n")



In [11]:
def apply_gamma(data):
    gamma = 0.35
    data = np.power(data, gamma)
    data *= 255 / data.max()
    data = data.astype("uint8")
    return data

assert os.path.isfile(f"../data/inputs/{s2_id}/sub1/rtc.txt"), "RTC file for the scene not ready or none-existent. Run `rtc_job_single_scene` notebook."
with open(f"../data/inputs/{s2_id}/sub1/rtc.txt", "r") as f:
    rtc_file = f.readline()

with ZipFile(rtc_file) as f:
    f.extractall(f"../data/inputs/{s2_id}/sub1")
    
tgt_image_dir = os.path.join("../data/inputs/", s2_id, "sub1", f"{os.path.splitext(os.path.basename(rtc_file))[0]}")
tgt_image = list(filter(lambda f: ("_HH" in f) and f.endswith(".tif"), glob.glob(f"{tgt_image_dir}/*")))[0]
enhanced_tgt_image = tgt_image.replace(".tif", "_enhanced.tif")
if os.path.isfile(enhanced_tgt_image):
    os.remove(enhanced_tgt_image)
downsample_dataset(tgt_image, 1.0, enhanced_tgt_image, apply_gamma)
tgt_image = enhanced_tgt_image
# repreojected_tgt_path = os.path.join(os.path.dirname(tgt_image), os.path.basename(tgt_image).replace(".tiff", "-reprojected.tiff"))
# reproject_tif(tgt_image, repreojected_tgt_path, rasterio.open(ref_image).profile["crs"])
# tgt_image = repreojected_tgt_path

local_output = os.path.join("../data/outputs/", "L1C_SLC_AN_pair", f"{s2_id}_local.tiff")
global_output = os.path.join("../data/outputs/", "L1C_SLC_AN_pair", f"{s2_id}_global.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=ref_image,
    im_tgt=tgt_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=1,
    align_grids=True,
    max_iter = 10,
    max_shift = 25,
    ws = (500, 500)
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=ref_image,
    im_tgt=tgt_image,
    grid_res=25,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    # s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()

##### S2 L1C AN stack (downsampled)

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]
s2_an_l1c_secne_files = s2_an_l1c_r031_dict[id]

# tdf = s2_an_df[s2_an_df.ID == id]
# tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
# tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
# s2_an_l1c_secne_files = list(tdf.Path)

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)

ref_id = 15
ref_scene = s2_an_l1c_secne_files[ref_id]
with ZipFile(ref_scene) as f:
        f.extractall(f"../data/inputs/{id}/ref")    
ref_image_dir = os.path.join("../data/inputs/", id, "ref")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

downsample_dataset(ref_image, 0.1, ref_image.replace(".jp2", "_ds.jp2"))
ref_image = ref_image.replace(".jp2", "_ds.jp2")

tgt_scenes = [s2_an_l1c_secne_files[i] for i in range(0, len(s2_an_l1c_secne_files)) if i != ref_id]
tgt_images = []
local_outputs = []
global_outputs = []
for i, zip_file_path in enumerate(tgt_scenes):
    print(f"Extracting {i + 1} of {len(tgt_scenes)}: {zip_file_path}")
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/tgt{i}")
    tgt_image_dir = os.path.join("../data/inputs/", id, f"tgt{i}")
    tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
    tgt_tci_files = list(filter(lambda f: "L1C" in f, tgt_tci_files))
    if len(tgt_tci_files) > 1:
        tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
    else:
        tgt_image = tgt_tci_files[0]
    tgt_images.append(tgt_image)
    local_outputs.append(os.path.join("../data/outputs/", "L1C_AN_Serie", f"{id}_local_{i}.tiff"))
    global_outputs.append(os.path.join("../data/outputs/", "L1C_AN_Serie", f"{id}_global_{i}.tiff"))

for img in tgt_images:
     downsample_dataset(img, 0.1, img.replace(".jp2", "_ds.jp2"))
tgt_images = [img.replace(".jp2", "_ds.jp2") for img in tgt_images]


In [ ]:
print(f"Reference image: {ref_image}")
processed_tgt_images = []
for i, tgt_image in enumerate(tgt_images):
    print(f"Coregistering {tgt_image}")
    coreg_global = COREG(
        im_ref=ref_image,
        im_tgt=tgt_image,
        path_out=global_outputs[i],
        fmt_out="GTIFF",
        # v=True,
        nodata=(0.0,0.0),
        r_b4match=2,
        s_b4match=2,
        align_grids=True,
        max_iter = 10,
        max_shift = 10,
        ws = (500, 500),
        ignore_errors=True,
    )
    res = coreg_global.correct_shifts()
    if not coreg_global.success:
        print(f"Coregistration not successfull for {tgt_image}. Removing the corresponding output: {global_outputs[i]}")
        os.remove(global_outputs[i])
    else:
        processed_tgt_images.append(tgt_image)

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_coreg_global.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
output_tgt_images = glob.glob("../data/outputs/L1C_AN_Serie/*")
output_tgt_images = [f for f in output_tgt_images if ("global" in f) and f.endswith(".tiff")]
fids = sorted([int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1]) for tgt in output_tgt_images])
output_tgt_images = [f"{os.path.splitext(output_tgt_images[0])[0].rsplit("_", maxsplit=1)[0]}_{id}.tiff" for id in fids]
datasets_paths = [ref_image] +  output_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_raw_global.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] +  processed_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
processed_tgt_images = []
print(f"Reference image: {ref_image}")
for i, tgt_image in enumerate(tgt_images):
    print(f"Coregistering {tgt_image}")
    coreg_local = COREG_LOCAL(
        im_ref=ref_image,
        im_tgt=tgt_image,
        grid_res=50,
        # max_points=200,
        path_out=local_outputs[i],
        fmt_out="GTIFF",
        # v=True,
        nodata=(0.0,0.0),
        r_b4match=2,
        s_b4match=2,
        align_grids=True,
        max_iter = 10,
        max_shift = 10,
        CPUs = 8,
        ignore_errors=True,
    )
    res = coreg_local.correct_shifts()
    if not coreg_local.success:
        print(f"Coregistration not successfull for {tgt_image}. Removing the corresponding output: {local_outputs[i]}")
        if os.path.isfile(local_outputs[i]):
            os.remove(local_outputs[i])
    else:
        processed_tgt_images.append(tgt_image)

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_coreg_local.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
output_tgt_images = glob.glob("../data/outputs/L1C_AN_Serie/*")
output_tgt_images = [f for f in output_tgt_images if ("local" in f) and f.endswith(".tiff")]
fids = sorted([int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1]) for tgt in output_tgt_images])
output_tgt_images = [f"{os.path.splitext(output_tgt_images[0])[0].rsplit("_", maxsplit=1)[0]}_{id}.tiff" for id in fids]
datasets_paths = [ref_image] +  output_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_raw_local.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] +  processed_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

##### S2 L1C AN 2024 N0511_R031

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]

tdf = s2_an_df[s2_an_df.ID == id]
tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
s2_an_l1c_secne_files = list(tdf.Path)

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)

ref_id = 4
ref_scene = s2_an_l1c_secne_files[ref_id]
with ZipFile(ref_scene) as f:
        f.extractall(f"../data/inputs/{id}/ref")    
ref_image_dir = os.path.join("../data/inputs/", id, "ref")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

downsample_dataset(ref_image, 0.1, ref_image.replace(".jp2", "_ds.jp2"))
ref_image = ref_image.replace(".jp2", "_ds.jp2")

tgt_scenes = [s2_an_l1c_secne_files[i] for i in range(0, len(s2_an_l1c_secne_files)) if i != ref_id]
tgt_images = []
local_outputs = []
global_outputs = []
for i, zip_file_path in enumerate(tgt_scenes):
    print(f"Extracting {i + 1} of {len(tgt_scenes)}: {zip_file_path}")
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/tgt{i}")
    tgt_image_dir = os.path.join("../data/inputs/", id, f"tgt{i}")
    tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
    tgt_tci_files = list(filter(lambda f: "L1C" in f, tgt_tci_files))
    if len(tgt_tci_files) > 1:
        tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
    else:
        tgt_image = tgt_tci_files[0]
    tgt_images.append(tgt_image)
    local_outputs.append(os.path.join("../data/outputs/", "L1C_AN_Serie", f"{id}_local_{i}.tiff"))
    global_outputs.append(os.path.join("../data/outputs/", "L1C_AN_Serie", f"{id}_global_{i}.tiff"))

for img in tgt_images:
     downsample_dataset(img, 0.1, img.replace(".jp2", "_ds.jp2"))
tgt_images = [img.replace(".jp2", "_ds.jp2") for img in tgt_images]

In [ ]:
print(f"Reference image: {ref_image}")
processed_tgt_images = []
for i, tgt_image in enumerate(tgt_images):
    print(f"Coregistering {tgt_image}")
    coreg_global = COREG(
        im_ref=ref_image,
        im_tgt=tgt_image,
        path_out=global_outputs[i],
        fmt_out="GTIFF",
        # v=True,
        nodata=(0.0,0.0),
        r_b4match=2,
        s_b4match=2,
        align_grids=True,
        max_iter = 10,
        max_shift = 10,
        ws = (500, 500),
        ignore_errors=True,
    )
    res = coreg_global.correct_shifts()
    if not coreg_global.success:
        print(f"Coregistration not successfull for {tgt_image}. Removing the corresponding output: {global_outputs[i]}")
        os.remove(global_outputs[i])
    else:
        processed_tgt_images.append(tgt_image)

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_coreg_global.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
output_tgt_images = glob.glob("../data/outputs/L1C_AN_Serie/*")
output_tgt_images = [f for f in output_tgt_images if ("global" in f) and f.endswith(".tiff")]
fids = sorted([int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1]) for tgt in output_tgt_images])
output_tgt_images = [f"{os.path.splitext(output_tgt_images[0])[0].rsplit("_", maxsplit=1)[0]}_{id}.tiff" for id in fids]
datasets_paths = [ref_image] +  output_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_raw_global.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] +  processed_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
processed_tgt_images = []
print(f"Reference image: {ref_image}")
for i, tgt_image in enumerate(tgt_images):
    print(f"Coregistering {tgt_image}")
    coreg_local = COREG_LOCAL(
        im_ref=ref_image,
        im_tgt=tgt_image,
        grid_res=50,
        # max_points=200,
        path_out=local_outputs[i],
        fmt_out="GTIFF",
        # v=True,
        nodata=(0.0,0.0),
        r_b4match=2,
        s_b4match=2,
        align_grids=True,
        max_iter = 10,
        max_shift = 10,
        CPUs = 8,
        ignore_errors=True,
    )
    res = coreg_local.correct_shifts()
    if not coreg_local.success:
        print(f"Coregistration not successfull for {tgt_image}. Removing the corresponding output: {local_outputs[i]}")
        if os.path.isfile(local_outputs[i]):
            os.remove(local_outputs[i])
    else:
        processed_tgt_images.append(tgt_image)

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_coreg_local.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
output_tgt_images = glob.glob("../data/outputs/L1C_AN_Serie/*")
output_tgt_images = [f for f in output_tgt_images if ("local" in f) and f.endswith(".tiff")]
fids = sorted([int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1]) for tgt in output_tgt_images])
output_tgt_images = [f"{os.path.splitext(output_tgt_images[0])[0].rsplit("_", maxsplit=1)[0]}_{id}.tiff" for id in fids]
datasets_paths = [ref_image] +  output_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

In [ ]:
out_gif = "../data/outputs/L1C_AN_Serie/s2_l1c_an_series_raw_local.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] +  processed_tgt_images
datasets_titles = ["Reference"] + [f"target_{id}" for id in fids]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
Image(filename=out_gif, width = 400, height=400) 

##### Co-register an arbitrary transformed scene to a reference scene for testing

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]
tdf = s2_an_df[s2_an_df.ID == id]
tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
s2_an_l1c_secne_files = list(tdf.Path)
ref_id = 4
ref_scene = s2_an_l1c_secne_files[ref_id]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
with ZipFile(ref_scene) as f:
        f.extractall(f"../data/inputs/{id}/ref")    
ref_image_dir = os.path.join("../data/inputs/", id, "ref")

ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

print(f"Reference image: {ref_image}")

params = {
    "translation_x": 100.0,
    "translation_y": 50.0,
    "rotation_angle": 30.0,
    "scale": 0.75,
}

tgt_image_dir = os.path.join("../data/inputs/", id, "tgt")
os.makedirs(tgt_image_dir, exist_ok=True)

tgt_image = os.path.join(tgt_image_dir, "tgt.tif")
warp_affine_dataset(ref_image, tgt_image, **params)

_, (axb, axt) = plt.subplots(1,2, figsize=(10, 20))
show(downsample_dataset(ref_image, 0.1)[0], ax=axb, title="Reference scene")
show(downsample_dataset(tgt_image, 0.1)[0], ax=axt, title="Target scene")

In [ ]:
output_dir = "data/outputs/warp_test"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "global_coreg.tif")
coreg_global = COREG(
    im_ref=ref_image,
    im_tgt=tgt_image,
    path_out=output_path,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 5000,
    ws = (10000, 10000),
)
res = coreg_global.correct_shifts()

In [ ]:
output_dir = "data/outputs/warp_test"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "local_coreg.tif")
coreg_local = COREG_LOCAL(
    im_ref=ref_image,
    im_tgt=tgt_image,
    grid_res=500,
    # max_points=200,
    path_out=output_path,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 150,
    CPUs = 8,
)
res = coreg_local.correct_shifts()